In [2]:
import json

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import v2
from torchvision.transforms import functional

import torchvision.utils as utils
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from PIL import Image

%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
class TeamClassificationDataset(Dataset):
    def __init__(self, img_folder_path, bboxes_file_path, transform = None, debug_img_show = False, debug_teams_train = False):
        super(TeamClassificationDataset, self).__init__()
        self.img_folder_path = img_folder_path
        self.transform = transform
        self.debug_img_show = debug_img_show
        self.debug_teams_train = debug_teams_train

        self.bboxes = {}
        with open(bboxes_file_path) as bboxes_json:
            self.bboxes = json.load(bboxes_json)
            self.bboxes = list(self.bboxes.items())

    def __len__(self):
        return len(self.bboxes)

    def load_image(self, imgage_path):
        image = Image.open(imgage_path).convert('RGB')
        return image

    def __getitem__(self, index):
        img_path = f'{self.img_folder_path}\\{self.bboxes[index][0]}.jpeg'
        img = self.load_image(img_path)
        height = img.height
        width = img.width

        orig_img = img
        if self.transform:
            img = self.transform(img)

        # process bboxes
        team_list = []
        lable_list = []
        bboxes_list = []
        colors_list = []
        for id, player in self.bboxes[index][1].items():
            box = player['box']
            x = box[0] * width
            y = box[1] * height
            w = box[2] * width
            h = box[3] * height
            bboxes_list.append([x, y, w, h])

            if self.debug_teams_train is True:
                lable_list.append(id)

                team = player['team']
                team_list.append(int(team))
                if int(team):
                    colors_list.append('green')
                else:
                    colors_list.append('red')
            else:
                colors_list.append('blue')

        bboxes_tensor = torch.tensor(bboxes_list)
        bboxes_tensor_xyxy = torchvision.ops.box_convert(bboxes_tensor, 'xywh', 'xyxy')

        if self.debug_img_show:
            debug_img = functional.pil_to_tensor(orig_img)
            debug_img = utils.draw_bounding_boxes(debug_img, 
                                                  bboxes_tensor_xyxy,
                                                  width=2,
                                                  colors=colors_list,
                                                  labels=lable_list,
                                                  font='verdana.ttf',
                                                  font_size=20)
            debug_img = transforms.ToPILImage()(debug_img)
            debug_img.show()

        return {'image': img, 
                'bboxes_tensor': bboxes_tensor,
                'teams_list': team_list}

transform = v2.Compose([
                v2.PILToTensor(), # uint8 [0, 255]
                v2.ToDtype(dtype=torch.float32, scale=True)# , # float32, [0, 1]
                #v2.Normalize((0.5,), (0.5,)), # (img - mean) / std [-1, 1]
           ])

train_dataset = TeamClassificationDataset('team_classification_data\\frames',
                                          'team_classification_data\\bboxes.json',
                                           transform,
                                           debug_img_show = False,
                                           debug_teams_train = True)

train_dataloader = DataLoader(train_dataset)

In [137]:
def compare(vec, gt):
    res = [1 if g in vec else 0 for g in gt]
    return res

def list_to_team_numbers(team_member_list):
    team1_numbers = []
    team2_numbers = []

    for i in range(10):
        if team_member_list[i] == 0:
            team1_numbers.append(i)
        else:
            team2_numbers.append(i)

    return team1_numbers, team2_numbers

def calculate_accuracy(gt, predicted):
    gt_team1, gt_team2 = list_to_team_numbers(gt)
    pr_team1, pr_team2 = list_to_team_numbers(predicted)

    res1 = compare(pr_team1, gt_team1)
    res2 = compare(pr_team2, gt_team2)
    accuracy = sum(res1 + res2) / len(gt_team1 + gt_team2)

    if accuracy < 0.5:
        res1 = compare(pr_team1, gt_team2)
        res2 = compare(pr_team2, gt_team1)
        accuracy = sum(res1 + res2) / len(gt_team1 + gt_team2)

    # print(res1, res2, 'accuracy:', accuracy)
    return accuracy


Mean RGB

In [141]:
from torchvision.transforms.v2.functional import crop
from kmeans_pytorch import kmeans

accuracy = []

for data in train_dataloader:

    image = data['image']
    bboxes = data['bboxes_tensor']
    teams = torch.stack(data['teams_list'])
    teams = teams.squeeze(-1)

    image = image.squeeze(0)

    mean_list = []
    for bb in bboxes[0]:
        cropped = crop(image,
                    left   = int(bb[0]),
                    top    = int(bb[1]),
                    width  = int(bb[2]),
                    height = int(bb[3]))
        
        # cropped_im = transforms.ToPILImage()(cropped)
        # cropped_im.show()

        # center_cropped = center_crop(cropped, [50, 50])
        # center_cropped_im = transforms.ToPILImage()(center_cropped)
        # center_cropped_im.show()

        mean_list.append(torch.mean(cropped, dim=[1, 2]))

    mean = torch.stack(mean_list)

    cluster_ids_x, cluster_centers = kmeans(X=mean, num_clusters=2, distance='cosine')
    # print(cluster_ids_x)
    # print(teams)
    accuracy.append(calculate_accuracy(cluster_ids_x, teams))

print('')
print('Resulting accuracy:')
print(sum(accuracy)/len(accuracy))

running k-means on cpu..


[running kmeans]: 2it [00:00, 998.29it/s, center_shift=0.000000, iteration=2, tol=0.000100]


running k-means on cpu..


[running kmeans]: 2it [00:00, 666.13it/s, center_shift=0.000000, iteration=2, tol=0.000100]


running k-means on cpu..


...

running k-means on cpu..


[running kmeans]: 3it [00:00, 702.96it/s, center_shift=0.000000, iteration=3, tol=0.000100]


running k-means on cpu..


[running kmeans]: 3it [00:00, 855.57it/s, center_shift=0.000000, iteration=3, tol=0.000100]


Resulting accuracy:
0.6090000000000007


In [ ]:
Mean HSV

In [144]:
from kornia.color import rgb_to_hsv

from torchvision.transforms.v2.functional import crop
from kmeans_pytorch import kmeans

accuracy = []

for data in train_dataloader:

    image = data['image']
    bboxes = data['bboxes_tensor']
    teams = torch.stack(data['teams_list'])
    teams = teams.squeeze(-1)

    image = image.squeeze(0)
    image = rgb_to_hsv(image)

    mean_list = []
    for bb in bboxes[0]:
        cropped = crop(image,
                    left   = int(bb[0]),
                    top    = int(bb[1]),
                    width  = int(bb[2]),
                    height = int(bb[3]))


        mean_list.append(torch.mean(cropped, dim=[1, 2]))

    mean = torch.stack(mean_list)

    cluster_ids_x, cluster_centers = kmeans(X=mean, num_clusters=2, distance='cosine')
    # print(cluster_ids_x)
    # print(teams)
    accuracy.append(calculate_accuracy(cluster_ids_x, teams))

print('')
print('Resulting accuracy:')
print(sum(accuracy)/len(accuracy))

running k-means on cpu..


[running kmeans]: 2it [00:00, 1000.43it/s, center_shift=0.000000, iteration=2, tol=0.000100]


running k-means on cpu..


[running kmeans]: 4it [00:00, 696.21it/s, center_shift=0.000000, iteration=4, tol=0.000100]


running k-means on cpu..


...

running k-means on cpu..


[running kmeans]: 2it [00:00, 987.71it/s, center_shift=0.000000, iteration=2, tol=0.000100]


running k-means on cpu..


[running kmeans]: 2it [00:00, 1000.19it/s, center_shift=0.000000, iteration=2, tol=0.000100]


Resulting accuracy:
0.6130000000000007


Hist RGB

In [160]:
from torchvision.transforms.v2.functional import crop
from kmeans_pytorch import kmeans
import kornia

accuracy = []

for data in train_dataloader:

    image = data['image']
    bboxes = data['bboxes_tensor']
    teams = torch.stack(data['teams_list'])
    teams = teams.squeeze(-1)

    image = image.squeeze(0)

    mean_list = []
    for bb in bboxes[0]:
        h = bb[3]
        w = bb[2]
        cropped = crop(image,
                    left   = int(bb[0]),
                    top    = int(bb[1]),
                    width  = int(h),
                    height = int(w))
        
        bbox_area = 2 * h * w

        hist = kornia.enhance.image_histogram2d(cropped, n_bins=8)
        print(hist[0].shape, hist[1].shape)
        print(hist[0]/bbox_area)

        mean_list.append(torch.mean(cropped, dim=[1, 2]))

        

    # mean = torch.stack(mean_list)
    # cluster_ids_x, cluster_centers = kmeans(X=mean, num_clusters=2, distance='cosine')
    # accuracy.append(calculate_accuracy(cluster_ids_x, teams))
    break

# print('')
# print('Resulting accuracy:')
# print(sum(accuracy)/len(accuracy))

torch.Size([3, 8]) torch.Size([3, 8])
tensor([[0.2537, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2526, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2518, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
torch.Size([3, 8]) torch.Size([3, 8])
tensor([[0.2561, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2548, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2551, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
torch.Size([3, 8]) torch.Size([3, 8])
tensor([[0.2542, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2539, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2542, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
torch.Size([3, 8]) torch.Size([3, 8])
tensor([[0.2570, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2557, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2552, 0.0000, 0.0000